In [1]:
import os
os.chdir('../')

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F

#root_dir = 'outputs/sd-v1-4'
root_dir = '/data/archive/sd-v1-4'
#ref_dir = os.path.join(root_dir, 'uni_pc_bh2_steps200_scale7.5')
#ref_dir = os.path.join(root_dir, 'dpm_solver_v3_steps5_scale7.5')
ref_dir = os.path.join(root_dir, 'dpm_solver++_steps200_scale7.5')
models = ['uni_pc_bh1', 'uni_pc_bh2', 'dpm_solver_v3']
steps = [5, 10, 15, 20, 25, 30, 35, 40]
scales= [7.5]

for scale in scales:
    for step in steps:
        for model in models:
            comp_dir = os.path.join(root_dir, f'{model}_steps{step}_scale{scale}')
            if not os.path.exists(comp_dir):
                continue
                
            mse_list = []
            for i in range(1000):
                ref_file = os.path.join(ref_dir, f"{i}.pt")
                comp_file = os.path.join(comp_dir, f"{i}.pt")
                if not os.path.exists(ref_file) or not os.path.exists(comp_file):
                    continue
                ref_data = torch.load(ref_file)
                comp_data = torch.load(comp_file)
                mses = torch.mean((ref_data['image'] - comp_data['image']) ** 2, dim=[1, 2, 3])
                mse_list.append(mses)
            mses = torch.cat(mse_list, dim=0)
            print(comp_dir, f'N={len(mses)}')
            print(f"{torch.mean(mses).item():.3f}")
        print("")
            

/data/archive/sd-v1-4/uni_pc_bh1_steps5_scale7.5 N=5000
0.391
/data/archive/sd-v1-4/uni_pc_bh2_steps5_scale7.5 N=5000
0.351
/data/archive/sd-v1-4/dpm_solver_v3_steps5_scale7.5 N=5000
0.410

/data/archive/sd-v1-4/uni_pc_bh1_steps10_scale7.5 N=5000
0.249
/data/archive/sd-v1-4/uni_pc_bh2_steps10_scale7.5 N=5000
0.238


<function torch.nn.functional.mse_loss(input: torch.Tensor, target: torch.Tensor, size_average: Optional[bool] = None, reduce: Optional[bool] = None, reduction: str = 'mean', weight: Optional[torch.Tensor] = None) -> torch.Tensor>